In [23]:
# -*- coding: utf-8 -*-
import jieba, codecs, math
from time import localtime,strftime

# Use 自建詞庫
# jieba.load_userdict("jieba-userdict.txt")

In [24]:
# Load news dataset
inputFile = codecs.open('demo-news.txt', 'r', 'utf-8')
articles = []
for line in inputFile:
    articles.append(line.strip())
inputFile.close()

print (articles[0])

2018年上海MWC今天登場，光寶（2301）宣布聯手國際大廠意法半導體（STMicroelectronics），共同打造全系列通訊模組，整合Sigfox與藍牙技術，推出多元雙效模組，強化產品效能，目前已取得Sigfox官方認證，可立即投入商用市場。 吳松泉表示，上海MWC期間亮相的全系列模組WSG300S、WSG303S、WSG304S與WSG306S，皆採用意法半導體系統晶片與無線收發器，包括STM32系統單晶片、BlueNRG-1低功耗藍牙（BLE）系統單晶片（System-on-Chip, SoC）及S2-LP Sub-1GHz無線收發器，不僅產品設計符合地區規範，同時兼顧卓越品質與尺寸優勢。 其中，WSG304S雙效模組整合了Sigfox與藍牙技術，搭配意法半導體的MEMS感測器，並內建32位元處理器，具備高效處理能力，在各種使用情境下，為客戶提供雙重通訊選擇，可靈活套用於各種近場控制、遠端傳輸等創新聯網情境。至於應用地區，WSG300S能同時支援RCZ2與RCZ4通訊地區，涵蓋美洲（美國、墨西哥、巴西、哥倫比亞、阿根廷）與紐西蘭、澳洲、新加坡、台灣、香港等地；WSG303S、WSG304S、WSG306S則可運用於歐洲、中東（阿曼、伊朗）與南非等區域。


In [48]:
# Calculate term frequency
specialChar = u"「」『』【】《》。，?？!！/╱:：;；()（）"
listOfTF = []
segmentedArticles = []

for index in range(len(articles)):
    termFrequency = {}
    terms = jieba.cut(articles[index], cut_all = False)
    temp = []
    for term in terms:
        if term.strip() == "" or term in specialChar: continue
        temp.append(term)
        if term not in termFrequency: termFrequency[term] = 1
        else: termFrequency[term] += 1
    listOfTF.append(termFrequency)
    segmentedArticles.append(" ".join(temp))

# save the segmented articles for later use
outputFile = codecs.open('segmented.txt', 'w', 'utf-8')

for index in range(len(segmentedArticles)):
    outputFile.write(segmentedArticles[index] + "\n")
outputFile.close()
print ("DONE")

DONE


In [43]:
# Get document frequency
termToTotalDocumentOccurrence = {}
for termFrequency in listOfTF:
    for term in termFrequency:
        if term in termToTotalDocumentOccurrence: 
            termToTotalDocumentOccurrence[term] += 1
        else: termToTotalDocumentOccurrence[term] = 1

# Get TF-IDF score for each article
listOfTFIDF = []
for index in range(len(listOfTF)):
    TFIDF = {}
    for term in listOfTF[index]:
        score = listOfTF[index][term] * math.log10 \
                (len(articles) * 1.0 /termToTotalDocumentOccurrence[term])
        TFIDF[term] = score
    listOfTFIDF.append(TFIDF)


In [44]:
def getVectorLength(some_vector):
    sum = 0
    allKeys = some_vector.keys()
    for singleKey in allKeys:
        sum += some_vector[singleKey] ** 2
    return sum ** 0.5

def getCosineSimilarity(docVector1, docVector2):
    score = 0.0
    for word in docVector1:
        if word in docVector2: 
            score += docVector1[word] * docVector2[word]
    docVector1Length = getVectorLength(docVector1)
    docVector2Length = getVectorLength(docVector2)
    if docVector2Length > 0: return score / (docVector1Length * docVector2Length)
    else: return 0

In [45]:
test = "昨天富邦悍將在新人選秀會，一口氣選入10名球員，是4隊最多。富邦領隊蔡承儒說：「這些球員都是我們想要的，而且都值得培養，雖然還有自主培訓這個方式，但我們最後還是決定用選秀選入，也算是對他們的肯定。」 富邦在第2輪選中內野手王正棠、第3輪挑中旅外投手陳品學，蔡承儒表示，這2名球員都是即戰力球員，他說：「王正棠內野中線甚至三壘都可以守，可以給球員輪替的空間，增加我們的球員深度，而且打擊跟選球能力都很好。」 陳品學在選秀會前，都跟富邦自主練球，蔡承儒笑說：「他應該很想來我們球隊，有時候我來球場，也看到他自己來新莊練習。陳品學不用簽約金，在談約上可能會快一點，我們的規劃是認為會成為下半季的即戰力。」 富邦一口氣在選秀會補進10名球員，是否今年季後會有一波釋出潮？蔡承儒說：「也不一定選了10人，就要釋出10人，就算球員多一點，也是沒什麼關係，多囤積一些年輕球員也不錯。」"
testTerms = jieba.cut(test, cut_all = False)
testVector = {}
for testTerm in testTerms:
    if testTerm.strip() == "" or testTerm in specialChar: continue
    if testTerm not in testVector: testVector[testTerm] = 1
    else: testVector[testTerm] += 1

for index in range(len(listOfTFIDF)):
    similarityScore = getCosineSimilarity(testVector, listOfTFIDF[index])
    print (str(similarityScore) + "\t" + articles[index])

0.0168895604245814	2018年上海MWC今天登場，光寶（2301）宣布聯手國際大廠意法半導體（STMicroelectronics），共同打造全系列通訊模組，整合Sigfox與藍牙技術，推出多元雙效模組，強化產品效能，目前已取得Sigfox官方認證，可立即投入商用市場。 吳松泉表示，上海MWC期間亮相的全系列模組WSG300S、WSG303S、WSG304S與WSG306S，皆採用意法半導體系統晶片與無線收發器，包括STM32系統單晶片、BlueNRG-1低功耗藍牙（BLE）系統單晶片（System-on-Chip, SoC）及S2-LP Sub-1GHz無線收發器，不僅產品設計符合地區規範，同時兼顧卓越品質與尺寸優勢。 其中，WSG304S雙效模組整合了Sigfox與藍牙技術，搭配意法半導體的MEMS感測器，並內建32位元處理器，具備高效處理能力，在各種使用情境下，為客戶提供雙重通訊選擇，可靈活套用於各種近場控制、遠端傳輸等創新聯網情境。至於應用地區，WSG300S能同時支援RCZ2與RCZ4通訊地區，涵蓋美洲（美國、墨西哥、巴西、哥倫比亞、阿根廷）與紐西蘭、澳洲、新加坡、台灣、香港等地；WSG303S、WSG304S、WSG306S則可運用於歐洲、中東（阿曼、伊朗）與南非等區域。
0.05954379694969273	「刷卡儲值」到街口支付、歐付寶等電子支付錢包，金管會再鬆綁。金管會昨預告「電子支付機構使用者身分確認機制及交易限額管理辦法」修正草案，「第一類電子支付帳戶」將新增可以「儲值」，儲值來源可以是信用卡、或是銀行帳戶。目前電子支付帳戶，依照身分認證強弱分三大類，第一類身分認證最弱，是採手機加上身分證資料，過去只能做代收付款，代收付款限額是3萬元，且無法做儲值。金管會昨對第一類電子支付帳戶，開放兩大項目，一是新增「儲值」，儲值上限是一萬元，二是代收付款限額，若民眾有臨時支付需求，可以提高到10萬元，但全年交易限額仍是36萬元。官員說，因為透過存款帳戶轉帳或是刷卡儲值等方式，可以追查資金流向，考量相關風險可以控管，因此才放行。此舉將可增加電子支付帳戶的消費支付彈性且提升電支帳戶使用便利性，有助增加使用者。至於第二類電子支付帳戶，則是以手機加上金融支付工具做身分認證，代收付限額是每月30萬元，儲值則是5萬元，第三類身分認證強度最強，採臨櫃或

In [1]:
import logging
from gensim.models import word2vec

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
sentences = word2vec.LineSentence('segmented.txt')
model = word2vec.Word2Vec(sentences, size=300)

# 保存模型，供日後使用
model.save("pttWord2Vec.model")

# 模型讀取方式
# model = word2vec.Word2Vec.load("your_model_name")

2018-10-04 15:10:14,915 : INFO : collecting all words and their counts
2018-10-04 15:10:14,917 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-10-04 15:10:14,918 : INFO : collected 1095 word types from a corpus of 2055 raw words and 10 sentences
2018-10-04 15:10:14,920 : INFO : Loading a fresh vocabulary
2018-10-04 15:10:14,923 : INFO : effective_min_count=5 retains 72 unique words (6% of original 1095, drops 1023)
2018-10-04 15:10:14,923 : INFO : effective_min_count=5 leaves 628 word corpus (30% of original 2055, drops 1427)
2018-10-04 15:10:14,924 : INFO : deleting the raw counts dictionary of 1095 items
2018-10-04 15:10:14,926 : INFO : sample=0.001 downsamples 72 most-common words
2018-10-04 15:10:14,926 : INFO : downsampling leaves estimated 204 word corpus (32.6% of prior 628)
2018-10-04 15:10:14,927 : INFO : estimated required memory for 72 words and 300 dimensions: 208800 bytes
2018-10-04 15:10:14,928 : INFO : resetting layer weights
2018-10-04 15

In [2]:
print (model['世界'])

[ 7.80497910e-04  1.06670265e-03 -1.31102221e-03  6.73092552e-04
  1.16630072e-04 -1.58528658e-03  1.45856873e-04 -9.66102292e-04
 -8.40064080e-04 -6.04859903e-04  4.47454571e-04  1.45251432e-03
  1.43462676e-03 -8.54947313e-04  9.16351390e-04 -8.08433178e-05
 -3.50019982e-05 -1.35207444e-03  6.01757842e-04  9.37169243e-04
  6.96533127e-04  1.33261411e-03  4.80973307e-04 -1.03081041e-03
 -3.04491987e-04  1.03935308e-03  1.63675309e-03  1.01669726e-03
  5.90004085e-04 -1.38276606e-03  1.53164333e-03 -6.63598184e-04
 -1.75822095e-03 -4.28906147e-04  2.88558018e-04  1.24953338e-03
 -1.62829272e-03  7.37018476e-04  2.31952508e-04 -1.05246389e-03
 -6.32293115e-04 -1.26206852e-03 -1.21160538e-03 -1.08947075e-04
  5.54813363e-04 -1.72962388e-03 -5.54645841e-04  2.18490619e-04
 -4.87226323e-04  1.04029651e-03 -1.06744503e-03  6.42599945e-04
 -1.42968993e-03  3.55379074e-04  1.12055778e-03  3.67602537e-04
  4.38929070e-04 -6.35809498e-04 -1.34888745e-04  1.77622424e-03
  1.09455187e-03 -8.75782

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


In [3]:
print("相似詞前 100 排序")
res = model.most_similar('世界',topn = 100)
for item in res:
    print(item[0]+","+str(item[1]))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  
2018-10-04 15:10:20,735 : INFO : precomputing L2-norms of word weight vectors


相似詞前 100 排序
會,0.20158466696739197
機構,0.1393730193376541
支付,0.1285887509584427
8,0.12492131441831589
5G,0.11995314061641693
儲值,0.11554837226867676
高國麟,0.11407682299613953
從,0.10970176011323929
3,0.10261745005846024
數位,0.10247179865837097
是,0.09658145904541016
移動,0.096504345536232
到,0.09070290625095367
、,0.09069439768791199
與,0.08713461458683014
的,0.08645789325237274
在,0.08156436681747437
後,0.0804208368062973
應用,0.07584051787853241
以,0.06880386173725128
泰國,0.06588061898946762
桃田賢斗,0.06522839516401291
中,0.06197640299797058
泰銖,0.04890333116054535
4,0.04888824746012688
來,0.048047710210084915
包括,0.045999616384506226
李彬,0.040957532823085785
單場,0.038660161197185516
我,0.03855273127555847
美元,0.0371430329978466
2,0.03660852834582329
說,0.03648305684328079
身分,0.03192094713449478
都,0.030335498973727226
.,0.02908411994576454
上,0.028828009963035583
合作,0.025292115285992622
對,0.02498623915016651
VR,0.023257769644260406
電子,0.02033282443881035
帳戶,0.020332789048552513
印地安人,0.010733116418123245
和,0.00859405

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
